In [1]:
import psycopg2
import pandas as pd
import pandas.io.sql as sqlio
from config import database, user, password, host, port

con = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)
sql = "SELECT * FROM Players;"
all_df = sqlio.read_sql_query(sql, con)
con = None

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced


In [4]:
all_df.head()

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
0,Bobby Abreu,26,2000,PHI,NL,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332
1,Bobby Abreu,27,2001,PHI,NL,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664
2,Bobby Abreu,28,2002,PHI,NL,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000
3,Bobby Abreu,29,2003,PHI,NL,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668
4,Bobby Abreu,30,2004,PHI,NL,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332


In [5]:
MLB_df = all_df.loc[(all_df["league"] != "NL/AL/NL") & (all_df["league"] != "NL/AL/AL")]


In [6]:
MLB_df.loc[(all_df["league"] == "NL/AL/AL") | (all_df["league"] == "NL/AL/NL")]

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete


In [7]:
MLB_df = MLB_df.replace(to_replace = "NL/AL", value="AL/NL", regex=True)



In [8]:
MLB_df = MLB_df.replace({'league': {'NL': 0, 'AL': 1, 'AL/NL': 2}})
MLB_df.head()

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
0,Bobby Abreu,26,2000,PHI,0,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332
1,Bobby Abreu,27,2001,PHI,0,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664
2,Bobby Abreu,28,2002,PHI,0,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000
3,Bobby Abreu,29,2003,PHI,0,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668
4,Bobby Abreu,30,2004,PHI,0,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332


In [9]:
MLB_df = MLB_df.drop(["team"], axis=1)
MLB_df

,name,age,year,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
0,Bobby Abreu,26,2000,0,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332
1,Bobby Abreu,27,2001,0,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664
2,Bobby Abreu,28,2002,0,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000
3,Bobby Abreu,29,2003,0,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668
4,Bobby Abreu,30,2004,0,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332
...,...,...,...,...,...,...,...,...,...,...,...
2382,Barry Zito,31,2009,0,22045868,1.026738,2.444635,0,0.543210,98451427,42.857143
2383,Barry Zito,32,2010,0,21690091,1.065954,1.060084,0,0.567901,114698178,57.142857
2384,Barry Zito,33,2011,0,21026387,0.286988,-2.264907,2,0.530864,134339671,71.428570
2385,Barry Zito,34,2012,0,21156837,0.985740,0.071013,0,0.580247,91284072,85.714290


In [14]:
X = MLB_df.drop(["war_psp"], axis=1)
X = pd.get_dummies(X)

In [32]:
from sklearn.mixture import GaussianMixture
n_clusters=12
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_1 = model.aic(X)

In [33]:
n_clusters=11
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_2 = model.aic(X)

In [34]:
n_clusters=10
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_3 = model.aic(X)

In [35]:
n_clusters=9
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_4 = model.aic(X)

In [36]:
n_clusters=8
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_5 = model.aic(X)

In [54]:
n_clusters=7
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_6 = model.aic(X)


GaussianMixture(n_components=7)


In [38]:
n_clusters=6
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_7 = model.aic(X)

In [39]:
n_clusters=5
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_8 = model.aic(X)

In [40]:
n_clusters=4
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_9 = model.aic(X)

In [41]:
n_clusters=3
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_10 = model.aic(X)

In [42]:
n_clusters=2
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_11 = model.aic(X)

In [43]:
n_clusters=1
model = GaussianMixture(n_components=n_clusters, init_params='kmeans')
model.fit(X)
AIC_12 = model.aic(X)

In [51]:
AIC = [AIC_1, AIC_2, AIC_3, AIC_4, AIC_5, AIC_6, AIC_7, AIC_8, AIC_9, AIC_10, AIC_11, AIC_12]
AIC.index(min(AIC))

5

In [53]:
AIC_6

-8555638.601556357

In [ ]:
model